# Face Mask Detection - Prediction & Deployment
This notebook demonstrates how to use the trained model for predictions and deployment

In [ ]:
# Import required libraries
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
import os
from utils.helpers import preprocess_face, detect_mask_colors

print(f"TensorFlow version: {tf.__version__}")
print(f"OpenCV version: {cv2.__version__}")

In [ ]:
# Load the trained model
try:
    model = tf.keras.models.load_model('model/face_mask_model.h5')
    print("Model loaded successfully!")
    model.summary()
except Exception as e:
    print(f"Error loading model: {e}")
    print("Please run the training notebook first to create the model.")
    model = None

In [ ]:
# Load face detection cascade
face_cascade = cv2.CascadeClassifier('model/haarcascade_frontalface_default.xml')
if face_cascade.empty():
    # Fallback to OpenCV's built-in cascade
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')
    print("Using OpenCV's built-in face cascade")
else:
    print("Face cascade loaded successfully")

In [ ]:
# Face Mask Detection Pipeline
class FaceMaskDetector:
    def __init__(self, model, face_cascade):
        self.model = model
        self.face_cascade = face_cascade
        
    def detect_faces(self, image):
        """Detect faces in the image"""
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(
            gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30)
        )
        return faces
    
    def predict_mask(self, face_region):
        """Predict if face has mask using the trained model"""
        if self.model is None:
            # Fallback to color-based detection
            has_mask, confidence = detect_mask_colors(face_region)
            return confidence if has_mask else 1 - confidence
        
        # Preprocess face for model
        processed_face = preprocess_face(face_region)
        
        # Make prediction
        prediction = self.model.predict(processed_face, verbose=0)
        return prediction[0][0]
    
    def process_image(self, image_path):
        """Process an image and detect masks"""
        # Load image
        image = cv2.imread(image_path)
        if image is None:
            raise ValueError(f"Could not load image: {image_path}")
        
        # Detect faces
        faces = self.detect_faces(image)
        
        results = []
        for (x, y, w, h) in faces:
            # Extract face region
            face_region = image[y:y+h, x:x+w]
            
            # Predict mask
            mask_prob = self.predict_mask(face_region)
            
            # Determine label and color
            if mask_prob > 0.5:
                label = "Mask"
                confidence = mask_prob * 100
                color = (0, 255, 0)  # Green
            else:
                label = "No Mask"
                confidence = (1 - mask_prob) * 100
                color = (0, 0, 255)  # Red
            
            results.append({
                'bbox': (x, y, w, h),
                'label': label,
                'confidence': confidence,
                'color': color,
                'mask_probability': mask_prob
            })
        
        return image, results
    
    def draw_results(self, image, results):
        """Draw bounding boxes and labels on image"""
        result_image = image.copy()
        
        for result in results:
            x, y, w, h = result['bbox']
            label = result['label']
            confidence = result['confidence']
            color = result['color']
            
            # Draw bounding box
            cv2.rectangle(result_image, (x, y), (x+w, y+h), color, 2)
            
            # Draw label
            text = f"{label}: {confidence:.1f}%"
            (text_width, text_height), baseline = cv2.getTextSize(
                text, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2
            )
            
            # Background rectangle for text
            cv2.rectangle(result_image, (x, y-text_height-10), 
                         (x+text_width, y), color, -1)
            cv2.putText(result_image, text, (x, y-5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        return result_image

# Initialize detector
detector = FaceMaskDetector(model, face_cascade)
print("Face Mask Detector initialized!")

In [ ]:
# Create test images for demonstration
def create_test_images():
    """Create sample test images"""
    os.makedirs('test_images', exist_ok=True)
    
    # Create sample images with faces
    for i in range(3):
        # Create a face-like image
        img = np.random.randint(100, 200, (300, 300, 3), dtype=np.uint8)
        
        # Add face features
        # Face outline
        cv2.ellipse(img, (150, 150), (80, 100), 0, 0, 360, (200, 180, 160), -1)
        
        # Eyes
        cv2.circle(img, (120, 120), 15, (50, 50, 50), -1)
        cv2.circle(img, (180, 120), 15, (50, 50, 50), -1)
        cv2.circle(img, (120, 120), 8, (255, 255, 255), -1)
        cv2.circle(img, (180, 120), 8, (255, 255, 255), -1)
        
        # Nose
        cv2.circle(img, (150, 150), 8, (180, 150, 130), -1)
        
        # Mouth area
        cv2.ellipse(img, (150, 180), (20, 10), 0, 0, 180, (150, 100, 100), -1)
        
        # Add mask to some images
        if i % 2 == 0:  # Add mask to even numbered images
            mask_color = [(255, 255, 255), (100, 150, 255), (50, 50, 50)][i]
            cv2.rectangle(img, (100, 160), (200, 200), mask_color, -1)
            # Add mask straps
            cv2.line(img, (100, 170), (80, 140), mask_color, 3)
            cv2.line(img, (200, 170), (220, 140), mask_color, 3)
        
        # Save image
        filename = f'test_images/test_face_{i+1}.jpg'
        cv2.imwrite(filename, img)
        print(f"Created {filename}")

# Create test images
create_test_images()

In [ ]:
# Test the detector on sample images
test_image_paths = [
    'test_images/test_face_1.jpg',
    'test_images/test_face_2.jpg',
    'test_images/test_face_3.jpg'
]

fig, axes = plt.subplots(2, 3, figsize=(15, 10))
fig.suptitle('Face Mask Detection Results', fontsize=16)

for i, image_path in enumerate(test_image_paths):
    if os.path.exists(image_path):
        # Process image
        original_image, results = detector.process_image(image_path)
        result_image = detector.draw_results(original_image, results)
        
        # Convert BGR to RGB for matplotlib
        original_rgb = cv2.cvtColor(original_image, cv2.COLOR_BGR2RGB)
        result_rgb = cv2.cvtColor(result_image, cv2.COLOR_BGR2RGB)
        
        # Display original image
        axes[0, i].imshow(original_rgb)
        axes[0, i].set_title(f'Original Image {i+1}')
        axes[0, i].axis('off')
        
        # Display result image
        axes[1, i].imshow(result_rgb)
        
        # Create title with detection results
        if results:
            result_text = ', '.join([f"{r['label']} ({r['confidence']:.1f}%)" for r in results])
            axes[1, i].set_title(f'Results: {result_text}')
        else:
            axes[1, i].set_title('No faces detected')
        
        axes[1, i].axis('off')
        
        # Print detailed results
        print(f"\nImage {i+1} Results:")
        print(f"Faces detected: {len(results)}")
        for j, result in enumerate(results):
            print(f"  Face {j+1}: {result['label']} (Confidence: {result['confidence']:.1f}%, Mask Prob: {result['mask_probability']:.3f})")

plt.tight_layout()
plt.show()

In [ ]:
# Real-time webcam detection (optional)
def run_webcam_detection():
    """Run real-time face mask detection using webcam"""
    cap = cv2.VideoCapture(0)
    
    if not cap.isOpened():
        print("Error: Could not open webcam")
        return
    
    print("Starting webcam detection. Press 'q' to quit.")
    
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        
        # Detect faces
        faces = detector.detect_faces(frame)
        
        # Process each face
        for (x, y, w, h) in faces:
            face_region = frame[y:y+h, x:x+w]
            mask_prob = detector.predict_mask(face_region)
            
            # Determine label and color
            if mask_prob > 0.5:
                label = "Mask"
                confidence = mask_prob * 100
                color = (0, 255, 0)
            else:
                label = "No Mask"
                confidence = (1 - mask_prob) * 100
                color = (0, 0, 255)
            
            # Draw bounding box and label
            cv2.rectangle(frame, (x, y), (x+w, y+h), color, 2)
            text = f"{label}: {confidence:.1f}%"
            cv2.putText(frame, text, (x, y-10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)
        
        # Display frame
        cv2.imshow('Face Mask Detection', frame)
        
        # Check for quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    
    cap.release()
    cv2.destroyAllWindows()
    print("Webcam detection stopped.")

# Uncomment the line below to run webcam detection
# run_webcam_detection()

In [ ]:
# Model deployment preparation
def prepare_for_deployment():
    """Prepare model for deployment"""
    print("Preparing model for deployment...")
    
    # Save model in different formats
    if model is not None:
        # TensorFlow SavedModel format (recommended for deployment)
        model.save('model/face_mask_model_deployment', save_format='tf')
        print("✓ Saved in TensorFlow SavedModel format")
        
        # TensorFlow Lite format (for mobile deployment)
        converter = tf.lite.TFLiteConverter.from_keras_model(model)
        tflite_model = converter.convert()
        
        with open('model/face_mask_model.tflite', 'wb') as f:
            f.write(tflite_model)
        print("✓ Saved in TensorFlow Lite format")
        
        # Model info
        print(f"\nModel Information:")
        print(f"Input shape: {model.input_shape}")
        print(f"Output shape: {model.output_shape}")
        print(f"Total parameters: {model.count_params():,}")
        
        # Create deployment script
        deployment_script = '''
# Face Mask Detection Deployment Script
import tensorflow as tf
import cv2
import numpy as np

class FaceMaskPredictor:
    def __init__(self, model_path):
        self.model = tf.keras.models.load_model(model_path)
        self.face_cascade = cv2.CascadeClassifier(
            cv2.data.haarcascades + 'haarcascade_frontalface_default.xml'
        )
    
    def predict(self, image_path):
        image = cv2.imread(image_path)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        faces = self.face_cascade.detectMultiScale(gray, 1.1, 5)
        
        results = []
        for (x, y, w, h) in faces:
            face = image[y:y+h, x:x+w]
            face_resized = cv2.resize(face, (150, 150)) / 255.0
            face_batch = np.expand_dims(face_resized, axis=0)
            
            prediction = self.model.predict(face_batch)[0][0]
            label = "Mask" if prediction > 0.5 else "No Mask"
            confidence = prediction if prediction > 0.5 else 1 - prediction
            
            results.append({
                'bbox': (x, y, w, h),
                'label': label,
                'confidence': confidence * 100
            })
        
        return results

# Usage example:
# predictor = FaceMaskPredictor('model/face_mask_model.h5')
# results = predictor.predict('path/to/image.jpg')
# print(results)
'''
        
        with open('deploy_model.py', 'w') as f:
            f.write(deployment_script)
        print("✓ Created deployment script: deploy_model.py")
        
    else:
        print("❌ No model available for deployment. Please train the model first.")

# Prepare for deployment
prepare_for_deployment()

In [ ]:
# Performance benchmarking
import time

def benchmark_model():
    """Benchmark model performance"""
    if model is None:
        print("No model available for benchmarking")
        return
    
    print("Benchmarking model performance...")
    
    # Create test batch
    test_batch = np.random.random((32, 150, 150, 3)).astype(np.float32)
    
    # Warm up
    for _ in range(5):
        _ = model.predict(test_batch, verbose=0)
    
    # Benchmark
    times = []
    for _ in range(10):
        start_time = time.time()
        _ = model.predict(test_batch, verbose=0)
        end_time = time.time()
        times.append(end_time - start_time)
    
    avg_time = np.mean(times)
    fps = 32 / avg_time  # Images per second
    
    print(f"Average inference time: {avg_time:.4f} seconds")
    print(f"Throughput: {fps:.2f} images/second")
    print(f"Per image: {avg_time/32*1000:.2f} ms")

benchmark_model()

## Deployment Options

### 1. Streamlit Web App
```bash
streamlit run streamlit_app.py
```

### 2. Flask API
```python
from flask import Flask, request, jsonify
import tensorflow as tf

app = Flask(__name__)
model = tf.keras.models.load_model('model/face_mask_model.h5')

@app.route('/predict', methods=['POST'])
def predict():
    # Handle image upload and prediction
    pass
```

### 3. Docker Deployment
```dockerfile
FROM tensorflow/tensorflow:2.13.0
COPY . /app
WORKDIR /app
RUN pip install -r requirements.txt
CMD ["streamlit", "run", "streamlit_app.py"]
```

### 4. Cloud Deployment
- **Google Cloud AI Platform**
- **AWS SageMaker**
- **Azure ML**
- **Heroku**